# MVSX and MVSJ with Stories UI

This notebook demonstrates how to create MVSX (single state with assets) and MVSJ (multiple snapshots) files in TypeScript.

- **MVSX**: Single state with embedded assets (useful for bundling structure files)
- **MVSJ**: Multiple snapshots for creating stories/slideshows

In [5]:
import { createBuilder, molstarNotebook, validateStateTree, MVSX } from "../../molviewspec-ts/mod.ts";

## MVSX Example

MVSX format allows you to bundle structure files and other assets with your MolViewSpec state.
You can use local file references that are resolved from the assets dictionary.

In [6]:
const builder = createBuilder();

// Define assets that can be referenced by name
const assets = {
  "1cbs.cif": "https://files.wwpdb.org/download/1cbs.cif",
  // To use a local file, you would provide the file path or data URI
  // "1cbs.cif": "./local/path/to/1cbs.cif",
};

// Reference the asset by name in the builder
builder
  .download("1cbs.cif")
  .parse("mmcif")
  .modelStructure()
  .component("all")
  .representation("cartoon")
  .color("blue");

// In TypeScript, you would create an MVSX object with the state and assets
// For this demo, we'll just output the state
const state = builder.getState();

console.log("State:");
console.log(JSON.stringify(state, null, 2));
console.log("\nAssets:");
console.log(JSON.stringify(assets, null, 2));

const mvsx = new MVSX(state, assets);

// To save to file in TypeScript:
// await Deno.writeTextFile("./path/to/1cbs.mvsj", JSON.stringify(state, null, 2));

// To view in Mol* Stories app, you would need to implement viewer integration
await molstarNotebook(mvsx)

State:
{
  "kind": "single",
  "root": {
    "kind": "root",
    "children": [
      {
        "kind": "download",
        "params": {
          "url": "1cbs.cif"
        },
        "children": [
          {
            "kind": "parse",
            "params": {
              "format": "mmcif"
            },
            "children": [
              {
                "kind": "structure",
                "params": {
                  "type": "model"
                },
                "children": [
                  {
                    "kind": "component",
                    "params": {
                      "selector": "all"
                    },
                    "children": [
                      {
                        "kind": "representation",
                        "params": {
                          "type": "cartoon"
                        },
                        "children": [
                          {
                            "kind": "color",
                    

## MVSJ Example - Multiple Snapshots

MVSJ format allows you to create multiple snapshots (states) that can be viewed as a slideshow
or story in the Mol* Stories viewer.

In [13]:
import type { Snapshot } from "../../molviewspec-ts/molviewspec/nodes.ts";

// Helper function to create a snapshot for a PDB structure
function loadPdbSnapshot(id: string, color: string): Snapshot {
  const builder = createBuilder();
  
  builder
    .download(`https://files.wwpdb.org/download/${id}.cif`)
    .parse("mmcif")
    .modelStructure()
    .component("all")
    .representation("cartoon")
    .color(color);
  
  return builder.getSnapshot({
    title: id,
    description: `Simple view of PDB ID ${id}`,
    linger_duration_ms: 3000,
    transition_duration_ms: 1000,
  });
}

In [14]:
// Create multiple snapshots
const snapshots = [
  loadPdbSnapshot("1cbs", "blue"),
  loadPdbSnapshot("1tqn", "red"),
];

// In TypeScript, you would create a States object
const statesData = {
  kind: "multiple" as const,
  metadata: {
    title: "PDB Slideshow",
    description: "A slideshow of different PDB structures",
    timestamp: new Date().toISOString(),
    version: "1.8",
  },
  snapshots: snapshots,
};

console.log(JSON.stringify(statesData, null, 2));

// To save to file:
// await Deno.writeTextFile("./path/to/slideshow.mvsj", JSON.stringify(statesData, null, 2));

// To create HTML for Mol* Stories viewer:
await molstarNotebook(statesData)

{
  "kind": "multiple",
  "metadata": {
    "title": "PDB Slideshow",
    "description": "A slideshow of different PDB structures",
    "timestamp": "2025-12-29T16:27:56.276Z",
    "version": "1.8"
  },
  "snapshots": [
    {
      "root": {
        "kind": "root",
        "children": [
          {
            "kind": "download",
            "params": {
              "url": "https://files.wwpdb.org/download/1cbs.cif"
            },
            "children": [
              {
                "kind": "parse",
                "params": {
                  "format": "mmcif"
                },
                "children": [
                  {
                    "kind": "structure",
                    "params": {
                      "type": "model"
                    },
                    "children": [
                      {
                        "kind": "component",
                        "params": {
                          "selector": "all"
                        },
            

## Advanced Snapshot Example

Create a more complex story with multiple states and transitions.

In [17]:
const builder1 = createBuilder();

const struct1 = builder1
  .download('https://files.wwpdb.org/download/1cbs.cif')
  .parse('mmcif')
  .modelStructure();

struct1
  .component("polymer")
  .representation("cartoon")
  .color("blue");

// Save first state and continue
builder1.saveState({
  title: "Overview",
  description: "Full structure view",
  linger_duration_ms: 2000,
});

// Build second state
const struct2 = builder1
  .download('https://files.wwpdb.org/download/1cbs.cif')
  .parse('mmcif')
  .modelStructure();

struct2
  .component("ligand")
  .representation('ball_and_stick')
  .color("red");

// Get all states
const multipleStates = builder1.getStates({
  title: "Structure Story",
  description: "A story showing different views of the structure",
});

console.log(JSON.stringify(multipleStates, null, 2));
await molstarNotebook(multipleStates)

{
  "kind": "multiple",
  "metadata": {
    "title": "Structure Story",
    "description": "A story showing different views of the structure",
    "timestamp": "2025-12-29T16:28:29.060Z",
    "version": "1.8.1"
  },
  "snapshots": [
    {
      "root": {
        "kind": "root",
        "children": [
          {
            "kind": "download",
            "params": {
              "url": "https://files.wwpdb.org/download/1cbs.cif"
            },
            "children": [
              {
                "kind": "parse",
                "params": {
                  "format": "mmcif"
                },
                "children": [
                  {
                    "kind": "structure",
                    "params": {
                      "type": "model"
                    },
                    "children": [
                      {
                        "kind": "component",
                        "params": {
                          "selector": "polymer"
                      